In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

# --- Paramètres de connexion à la base de données ---
# (Remplace par tes propres informations)
db_user = 'root'
db_password = 'root'
db_host = 'localhost'
db_port = '3306'
db_name = 'sicda_easytime'

# Création du moteur de connexion SQLAlchemy
try:
    engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
    print("Connexion à la base de données réussie !")
except Exception as e:
    print(f"Erreur de connexion : {e}")

Connexion à la base de données réussie !


In [2]:
# --- Génération des données synthétiques ---

num_samples = 2000  # Nombre d'exemples à générer
data = []

for _ in range(num_samples):
    duree_retard_minutes = random.randint(1, 60)
    nb_retards_mois_precedent = random.randint(0, 8)
    charge_travail_equipe_jour = round(random.uniform(0.1, 1.0), 2)
    est_debut_semaine = random.choice([0, 1]) # 1 pour Lundi, 0 sinon

    # Règle métier pour la décision réelle (notre "vérité terrain")
    # On tolère plus facilement les petits retards si l'employé n'est pas un récidiviste
    # et si la charge de travail n'est pas critique.
    decision_reelle = 0  # 0 = Justification requise (par défaut)
    if duree_retard_minutes <= 15 and nb_retards_mois_precedent <= 2 and charge_travail_equipe_jour < 0.8:
        decision_reelle = 1 # 1 = Tolérer
    elif duree_retard_minutes <= 5: # On tolère toujours les très petits retards
        decision_reelle = 1

    data.append([
        duree_retard_minutes,
        nb_retards_mois_precedent,
        charge_travail_equipe_jour,
        est_debut_semaine,
        decision_reelle
    ])

# Création du DataFrame
columns = [
    'duree_retard_minutes', 
    'nb_retards_mois_precedent', 
    'charge_travail_equipe_jour', 
    'est_debut_semaine', 
    'decision_reelle'
]
df_retards = pd.DataFrame(data, columns=columns)

# Sauvegarde du dataset pour une utilisation future
df_retards.to_csv('dataset_retards_contextuel.csv', index=False)

print("Dataset généré et sauvegardé dans 'dataset_retards_contextuel.csv'")
df_retards.head()

Dataset généré et sauvegardé dans 'dataset_retards_contextuel.csv'


,duree_retard_minutes,nb_retards_mois_precedent,charge_travail_equipe_jour,est_debut_semaine,decision_reelle
0,1,7,0.56,0,1
1,30,7,0.31,0,0
2,3,3,0.38,0,1
3,30,5,0.96,0,0
4,27,8,0.66,1,0


In [3]:
# --- Entraînement du modèle ---

# Définition des features (X) et de la cible (y)
features = [
    'duree_retard_minutes', 
    'nb_retards_mois_precedent', 
    'charge_travail_equipe_jour', 
    'est_debut_semaine'
]
target = 'decision_reelle'

X = df_retards[features]
y = df_retards[target]

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Création et entraînement du modèle
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

print("Modèle entraîné avec succès !")

# Évaluation du modèle
y_pred = model.predict(X_test)
print("\nRapport de classification sur l'ensemble de test :")
print(classification_report(y_test, y_pred))

Modèle entraîné avec succès !

Rapport de classification sur l'ensemble de test :
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       354
           1       0.81      0.93      0.87        46

    accuracy                           0.97       400
   macro avg       0.90      0.95      0.93       400
weighted avg       0.97      0.97      0.97       400



In [4]:
# --- Sauvegarde du modèle entraîné ---

model_filename = 'model_retards_contextuel.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"\nLe modèle a été sauvegardé avec succès dans le fichier '{model_filename}'")


Le modèle a été sauvegardé avec succès dans le fichier 'model_retards_contextuel.pkl'
